# Compute gradient of OpenMM potenials

In [ ]:
# Import packages
import torch
from autograd import grad
from autograd import numpy as np
from openmmtools.constants import kB
from simtk import openmm as mm
from simtk import unit
from simtk.openmm import app
from openmmtools.testsystems import AlanineDipeptideImplicit

import boltzgen.openmm_interface as omi

In [ ]:
# Set up simulation object
temperature = 298
kT = kB * temperature

testsystem = AlanineDipeptideImplicit()
implicit_sim = app.Simulation(testsystem.topology,
                              testsystem.system,
                              mm.LangevinIntegrator(temperature * unit.kelvin , 1.0 / unit.picosecond, 1.0 * unit.femtosecond),
                              platform=mm.Platform.getPlatformByName('CPU')
                              )
implicit_sim.context.setPositions(testsystem.positions)

In [ ]:
openmm_energy = omi.OpenMMEnergyInterface.apply

In [ ]:
pos = torch.tensor(np.array(testsystem.positions)[None, ...], requires_grad=True)

In [ ]:
E = openmm_energy(pos, implicit_sim.context, temperature)

In [ ]:
E.backward()
print(pos.grad)

In [ ]:
# Check whether element [0, 0, 0] is equal to the numerical grad
h = 1e-4
pos[0, 0, 0] = pos[0, 0, 0] - h
E1 = openmm_energy(pos, implicit_sim.context, temperature)
pos[0, 0, 0] = pos[0, 0, 0] + 2 * h
E2 = openmm_energy(pos, implicit_sim.context, temperature)
pos[0, 0, 0] = pos[0, 0, 0] - h
print((E2 - E1) / (2 * h))